## Exploration of 100d space of genome vectors

Genome vectors created by the Dna2VecDataBunch exhibit piculiar patterns. This notebook is dedicated to exploratoin 
of the bacterial genome space using dimensionality reduction techniques

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.append("../mylib/")

from genomic import sequence
from genomic.sequence import regex_filter, count_filter
from functools import partial
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import manifold,neighbors
from scipy.cluster.hierarchy import dendrogram, linkage  
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(color_codes=True)
import plotly.plotly as py
import plotly.graph_objs as go

### Load Data

In [6]:
# DB="/data/genomes/GenSeq_fastas/train"
DB='/home/serge/development/genomes/ncbi-genomes-2019-04-07/bacterial genomes'

In [7]:
filters=[partial(regex_filter, rx="Escherichia|Klebsiella|Bacillus"),partial(regex_filter, rx="plasmid?\s", keep=False),
         partial(count_filter,num_fastas=(1,1), keep=1)]
data = sequence.Dna2VecList.from_folder(DB,filters=filters,agg=partial(np.mean, axis=0),n_cpus=7)

In [8]:
len(data.items)

1686

In [9]:
processors = [
    sequence.GSFileProcessor(),
    sequence.GSTokenizeProcessor(tokenizer=sequence.GSTokenizer(ngram=8, skip=0, n_cpus=4)),
    sequence.Dna2VecProcessor()]
%time for p in processors: p.process(data)

CPU times: user 42min 11s, sys: 8min 41s, total: 50min 52s
Wall time: 1h 25min 43s


In [10]:
len(data.items)

1686

###  Genome vectors

In [11]:
data.items

[array([-10600.021 ,  95022.27  ,  -4546.29  , -28514.387 , ..., -29253.547 , -42794.895 , -26871.03  ,  -7630.3984],
       dtype=float32),
 array([ -9848.35  , 132596.69  ,  -7269.2188, -46002.242 , ..., -16791.951 , -45528.703 , -51696.32  , -16984.848 ],
       dtype=float32),
 array([-11893.81 , 148440.22 ,  -8725.728, -52156.43 , ..., -19914.074, -50862.883, -56499.875, -17946.658],
       dtype=float32),
 array([-18210.059, 117143.05 , -11808.324, -37285.65 , ..., -43233.8  , -61256.727, -33373.516,  -5143.087],
       dtype=float32),
 array([-17936.713, 114599.05 , -10966.08 , -35446.773, ..., -41979.465, -59523.035, -32904.266,  -5278.633],
       dtype=float32),
 array([-17462.723 , 114580.625 , -11426.668 , -36419.715 , ..., -42048.332 , -59268.53  , -32817.168 ,  -5567.8833],
       dtype=float32),
 array([-18008.34 , 114250.88 , -11187.709, -35736.53 , ..., -41940.695, -59488.145, -32739.904,  -5188.623],
       dtype=float32),
 array([ -4641.001 , 117018.06  ,   5746.5435

In [14]:
def log_scale(X):
    x=np.asarray(X);e=1e-6
    return np.log10(x+np.abs(x.min())+e) 


x=np.asarray(data.items)
bad_fastas = np.where(np.mean(x,axis=1) == 0.)[0]
X = np.delete(x, bad_fastas,0)
labelList=[" ".join(i.split()[1:3]) for i in data.descriptions]
labelList=np.delete(np.asarray(labelList), bad_fastas)
vocab=list(np.unique(labelList))
y=[vocab.index(x) for x in labelList]

## Correlation Distance in log-scaled space

### tSNE

In [18]:
tsne = manifold.TSNE(n_components=3, init='pca', perplexity=10, metric="correlation",random_state=0)
%time X3 = tsne.fit_transform(log_scale(X))

genus = [i.split()[0] for i in labelList]
genus_vocab=list(np.unique(genus))
y=[genus_vocab.index(x) for x in genus]
genus_vocab

X3_df = pd.DataFrame(data=X3, columns=["pc1",'pc2','pc3'], index=labelList)

X3_df["genus"]=genus
X3_df["y"]=y

genus_df=X3_df.groupby("genus").agg({"pc1": list, "pc2":list,"pc3":list,"y":np.mean})

CPU times: user 31.1 s, sys: 313 ms, total: 31.4 s
Wall time: 30.9 s


### Correlation Distance visualisation

In [20]:
data=[]
for g in genus_df.index:
    trace  = go.Scatter3d(
        name = str(g),
        x=genus_df.loc[g,"pc1"],
        y=genus_df.loc[g,"pc2"],
        z=genus_df.loc[g,"pc3"],
        mode='markers',
        marker=dict(
            size=8,
            color=genus_df.loc[g,"y"],                # set color to an array/list of desired values
            colorscale='Jet',           # choose a colorscale
            opacity=0.5)
    )

    data.append(trace)
    

layout = go.Layout(
    width=1000,
    height=1000,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='correlation distance ncbi-genomes-2019-04-07 Escherichia,Klebsiella,Bacillus')

/home/serge/anaconda3/envs/bio/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



## Eucleadian Distance in log-scaled space

### tSNE

In [21]:
tsne = manifold.TSNE(n_components=3, init='pca', perplexity=30,random_state=0)
%time X3 = tsne.fit_transform(log_scale(X))

CPU times: user 25.8 s, sys: 334 ms, total: 26.1 s
Wall time: 25.5 s


In [22]:
genus = [i.split()[0] for i in labelList]
genus_vocab=list(np.unique(genus))
y=[genus_vocab.index(x) for x in genus]
genus_vocab

X3_df = pd.DataFrame(data=X3, columns=["pc1",'pc2','pc3'], index=labelList)

X3_df["genus"]=genus
X3_df["y"]=y

genus_df=X3_df.groupby("genus").agg({"pc1": list, "pc2":list,"pc3":list,"y":np.mean})

### Eucleadian Distance Visualisation

In [32]:
data=[]
for g in genus_df.index:
    trace  = go.Scatter3d(
        name = str(g),
        x=genus_df.loc[g,"pc1"],
        y=genus_df.loc[g,"pc2"],
        z=genus_df.loc[g,"pc3"],
        mode='markers',
        marker=dict(
            size=8,
            color=genus_df.loc[g,"y"]+1,                # set color to an array/list of desired values
            colorscale='YlGnBu',           # choose a colorscale
            opacity=0.5)
    )

    data.append(trace)
    

layout = go.Layout(
    width=1000,
    height=1000,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='eucledian distance metric by genus Escherichia|Klebsiella|Bacillus')

/home/serge/anaconda3/envs/bio/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



## Eucleadian Distance in unmodified space

### tSNE

In [27]:
tsne = manifold.TSNE(n_components=3, init='pca', perplexity=30,random_state=0)
%time X3 = tsne.fit_transform(X)

CPU times: user 41.5 s, sys: 313 ms, total: 41.8 s
Wall time: 41.2 s


In [28]:
genus = [i.split()[0] for i in labelList]
genus_vocab=list(np.unique(genus))
y=[genus_vocab.index(x) for x in genus]
genus_vocab

X3_df = pd.DataFrame(data=X3, columns=["pc1",'pc2','pc3'], index=labelList)

X3_df["genus"]=genus
X3_df["y"]=y

genus_df=X3_df.groupby("genus").agg({"pc1": list, "pc2":list,"pc3":list,"y":np.mean})

### Eucleadian Distance Visualisation

In [29]:
data=[]
for g in genus_df.index:
    trace  = go.Scatter3d(
        name = str(g),
        x=genus_df.loc[g,"pc1"],
        y=genus_df.loc[g,"pc2"],
        z=genus_df.loc[g,"pc3"],
        mode='markers',
        marker=dict(
            size=8,
            color=genus_df.loc[g,"y"],                # set color to an array/list of desired values
            colorscale='Jet',           # choose a colorscale
            opacity=0.5)
    )

    data.append(trace)
    

layout = go.Layout(
    width=1000,
    height=1000,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='eucledian distance in native space Escherichia|Klebsiella|Bacillus')

/home/serge/anaconda3/envs/bio/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



## Genome Inventory

In [8]:
all_fastas = sequence.Dna2VecList.from_folder(DB)

In [28]:
next(iter(all_fastas))

{'file': '/home/serge/development/genomes/ncbi-genomes-2019-04-07/bacterial genomes/GCF_003261035.1_ASM326103v1_genomic.fna',
 'description': 'NZ_CP030062.1 Mannheimia varigena strain Teagasc 1 chromosome, complete genome',
 'id': 'NZ_CP030062.1',
 'name': 'NZ_CP030062.1'}

In [30]:
a = [x for x in all_fastas if "plasmid" not in x['description'].split()] 

In [31]:
a

[{'file': '/home/serge/development/genomes/ncbi-genomes-2019-04-07/bacterial genomes/GCF_003261035.1_ASM326103v1_genomic.fna',
  'description': 'NZ_CP030062.1 Mannheimia varigena strain Teagasc 1 chromosome, complete genome',
  'id': 'NZ_CP030062.1',
  'name': 'NZ_CP030062.1'},
 {'file': '/home/serge/development/genomes/ncbi-genomes-2019-04-07/bacterial genomes/GCF_000750395.2_ASM75039v2_genomic.fna',
  'description': 'NZ_CP009085.2 Salmonella enterica subsp. enterica serovar Enteritidis strain OLF-SE3-98983-4, complete genome',
  'id': 'NZ_CP009085.2',
  'name': 'NZ_CP009085.2'},
 {'file': '/home/serge/development/genomes/ncbi-genomes-2019-04-07/bacterial genomes/GCF_900205745.1_Lf_IMDO_130101_genomic.fna',
  'description': 'NZ_LT906621.1 Lactobacillus fermentum strain IMDO 130101 genome assembly, chromosome: I',
  'id': 'NZ_LT906621.1',
  'name': 'NZ_LT906621.1'},
 {'file': '/home/serge/development/genomes/ncbi-genomes-2019-04-07/bacterial genomes/GCF_001854145.2_ASM185414v2_genomic.

In [25]:
a = pd.DataFrame(data=list(a.items), columns=["file", "description","id","name"]).groupby("file").agg({"id":'count'}).sort_values('id', ascending=False)

AttributeError: 'list' object has no attribute 'items'

In [20]:
a.describe()

,id
count,13050.000000
mean,1.956552
std,1.742139
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,22.000000


In [26]:
inventory = pd.DataFrame(data=[l.split()[1:3] for l in all_fastas], columns=["genus","species" ])

In [30]:
inventory.groupby("genus").agg({"species":"count"}).sort_values("species",ascending=False)

,species
genus,
Escherichia,2239
Klebsiella,1718
Salmonella,1183
Bacillus,1172
Lactobacillus,953
Staphylococcus,889
Burkholderia,650
Enterococcus,626
Pseudomonas,613


In [6]:
all_fasts

NameError: name 'all_fasts' is not defined

In [24]:
inventory.groupby(["genus", "species"]).agg({"species": "count"})
inventory.columns=["count"]
inventory

count
genus            species                
'Catharanthus    roseus'               2
'Deinococcus     soli'                 1
'Nostoc          azollae'              3
18,711,729       reads                 1
Acaryochloris    marina               10
Acetobacter      aceti                 1
                 ascendens             1
                 orientalis            2
                 oryzifermentans       1
                 pasteurianus         91
                 persici               2
                 pomorum               7
                 senegalensis          2
                 sp.                   7
                 tropicalis            2
Acetobacterium   woodii                1
Acetohalobium    arabaticum            1
Acetomicrobium   mobile                1
Acholeplasma     axanthum              1
                 hippikon              2
                 laidlawii             2
                 oculi                 1
Achromobacter    denitrificans         3
                 insolitus             4
                 sp.                   2
                 spanius               4
                 xylosoxidans         11
Acidaminococcus  fermentans            1
                 intestini             1
Acidiferrobacter sp.                   1
...                                  ...
Zymobacter       palmae                2
Zymomonas        mobilis              49
[Arcobacter]     porcinus              1
[Bacillus        thuringiensis]        2
[Bacillus]       caldolyticus          2
                 selenitireducens      1
[Brevibacterium] flavum                2
                 frigoritolerans       1
[Clostridium]    bolteae               2
                 cellulolyticum        1
                 cellulosi             1
                 clariflavum           1
                 propionicum           1
                 saccharolyticum       1
                 scindens              1
                 stercorarium          4
[Enterobacter]   lignolyticus          1
[Eubacterium]    eligens               3
                 hallii                1
                 rectale               1
[Haemophilus]    ducreyi              14
                 parasuis              4
[Mycobacterium]  chelonae              2
[Pasteurella]    aerogenes             1
[Polyangium]     brachysporum          1
[Pseudomonas     syringae]             3
[Pseudomonas]    mesoacidophila        4
complete         chromosome            2
gamma            proteobacterium       1
secondary        endosymbiont          1

[3186 rows x 1 columns]

In [25]:
all_fastas

['NZ_CP013305.1 Vibrio cholerae strain CRC1106 chromosome 1, complete sequence',
 'NZ_CP013306.1 Vibrio cholerae strain CRC1106 chromosome 2, complete sequence',
 'NC_013791.2 Bacillus pseudofirmus OF4, complete genome',
 'NC_013792.1 Bacillus pseudofirmus OF4 plasmid pBpOF4-01, complete sequence',
 'NC_013793.1 Bacillus pseudofirmus OF4 plasmid pBpOF4-02, complete sequence',
 'NC_000913.3 Escherichia coli str. K-12 substr. MG1655, complete genome',
 'NC_007164.1 Corynebacterium jeikeium K411 complete genome',
 'NC_003080.1 Corynebacterium jeikeium K411 plasmid pKW4, complete sequence',
 'NC_002162.1 Ureaplasma parvum serovar 3 str. ATCC 700970, complete genome',
 'NC_004088.1 Yersinia pestis KIM10+, complete genome',
 'NC_004838.1 Yersinia pestis KIM10+ plasmid pMT-1, complete sequence',
 'NC_002620.2 Chlamydia muridarum Nigg, complete genome',
 'NC_002182.1 Chlamydia muridarum Nigg plasmid pMoPn, complete sequence',
 'NC_002488.3 Xylella fastidiosa 9a5c, complete genome',
 'NC_002489

In [117]:
counts = inventory.reset_index().groupby("genus").agg({"count", sum}).drop(("species"), axis=1)
counts.columns=["n_sequences","species"]
counts.sort_values("n_sequences", ascending=False)

,n_sequences,species
genus,,
Bacillus,1132,11
Streptomyces,743,5
Vibrio,468,5
Rhizobium,325,6
Pseudomonas,304,8
Staphylococcus,301,6
Clostridium,259,5
Streptococcus,222,6
Planktothrix,179,5
